# Examen 2do bimestre
Alexis Guanoluisa

## Obtención del corpus

In [ ]:
# Instala la librería 'datasets'
!pip install datasets

In [ ]:
# Configurar el token de Kaggle en Colab
import os
from google.colab import files

In [ ]:
# Pide subir el archivo kaggle.json
print("Por favor, sube tu archivo 'kaggle.json' descargado desde tu cuenta de Kaggle.")
uploaded = files.upload()

# Verificar que el archivo se subió correctamente
if 'kaggle.json' in uploaded:
    print("Archivo 'kaggle.json' subido correctamente.")

    # Crear el directorio .kaggle y mover el archivo
    if not os.path.exists('/root/.kaggle'):
        os.makedirs('/root/.kaggle')

    # Mover el archivo al directorio correcto
    os.rename('kaggle.json', '/root/.kaggle/kaggle.json')

    # Asignar los permisos correctos al archivo
    os.chmod('/root/.kaggle/kaggle.json', 600)

else:
    print("Error: No se encontró 'kaggle.json'. Por favor, asegúrate de subir el archivo correcto.")

Por favor, sube tu archivo 'kaggle.json' descargado desde tu cuenta de Kaggle.


Saving kaggle.json to kaggle.json
Archivo 'kaggle.json' subido correctamente.


In [ ]:
# Descargar el dataset usando kagglehub
if os.path.exists('/root/.kaggle/kaggle.json'):
    import kagglehub

    print("\nAutenticación exitosa. Descargando el dataset...")

    # Descargar la última versión del dataset
    path = kagglehub.dataset_download("Cornell-University/arxiv")

    print(f"\n¡Descarga completada! Path a los archivos del dataset: {path}")
else:
    print("\nLa descarga no puede continuar debido a un error de autenticación.")


Autenticación exitosa. Descargando el dataset...

¡Descarga completada! Path a los archivos del dataset: /kaggle/input/arxiv


In [ ]:
import zipfile
import json

# Crear el Subset
source_json_path = os.path.join("/kaggle/input/arxiv", 'arxiv-metadata-oai-snapshot.json')
subset_json_path = 'arxiv_subset.json'
subset_size = 20000
documents_subset = []

print(f"Leyendo el archivo fuente para crear un subset de {subset_size} documentos...")
with open(source_json_path, 'r', encoding='utf-8') as f:
    # Leer línea por línea para no cargar todo el archivo en memoria
    for i, line in enumerate(f):
        if i >= subset_size:
            break
        documents_subset.append(json.loads(line))

# Guardar nuestro subset en un nuevo archivo JSON
with open(subset_json_path, 'w', encoding='utf-8') as f:
    json.dump(documents_subset, f, ensure_ascii=False, indent=4)

print(f"Subset de {len(documents_subset)} documentos guardado en '{subset_json_path}'.")

Leyendo el archivo fuente para crear un subset de 20000 documentos...
Subset de 20000 documentos guardado en 'arxiv_subset.json'.


## Preprocesamiento

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import json
import re
import pandas as pd

In [ ]:
def preprocesar_texto(texto: str) -> str:

    if not texto:
        return ""
    texto = re.sub(r'-\n([a-z])', r'\1', texto)
    texto = texto.replace("\n", " ").replace("\t", " ")
    texto = re.sub(r'[^a-zA-Z0-9 ]', ' ', texto)
    texto = texto.lower()
    tokens = word_tokenize(texto)
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return " ".join(tokens)

In [ ]:
# Cargar datos y crear DataFrame
input_file = 'arxiv_subset.json'
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Crear el DataFrame
df = pd.DataFrame(data)
df = df[['id', 'title', 'abstract']]

In [ ]:
df

,id,title,abstract
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...
...,...,...,...
19995,0708.2144,Some Applications of Ricci Flow in Physics,I discuss certain applications of the Ricci ...
19996,0708.2145,Results and Frontiers in Lattice Baryon Spectr...,The Lattice Hadron Physics Collaboration (LH...
19997,0708.2146,Two-Fermion Bound States within the Bethe-Salp...,To solve the spinor-spinor Bethe-Salpeter eq...
19998,0708.2147,The logistic equation and a critique of the th...,Species coexistence is one of the central th...


In [ ]:
# Combinar título y resumen para un contexto más rico
df['texto_preprocesado'] = df['title'] + " " + df['abstract']
df['texto_preprocesado'] = df['texto_preprocesado'].apply(preprocesar_texto)

In [ ]:
df.to_pickle('preprocessed_dataframe.pkl')

## Indexación TF-IDF

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df['texto_preprocesado']

# Crear y entrenar el Vectorizer TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, max_df=0.85, min_df=2)
tfidf_matrix = vectorizer.fit_transform(corpus)

print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print("Índice TF-IDF creado exitosamente.")

# Guardar los objetos para la fase de búsqueda
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

Dimensiones de la matriz TF-IDF: (20000, 10000)
Índice TF-IDF creado exitosamente.


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los objetos necesarios
print("Cargando índice TF-IDF y DataFrame...")
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

with open('tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

df = pd.read_pickle('preprocessed_dataframe.pkl')
print(" Archivos cargados.")

Cargando índice TF-IDF y DataFrame...
 Archivos cargados.


In [ ]:
# Función de Búsqueda TF-IDF
def search_tfidf(query: str, top_k: int = 10):
    """
    Busca los top_k documentos más relevantes para una consulta usando el índice TF-IDF.
    """
    # Preprocesar la consulta
    query_processed = preprocesar_texto(query)

    # Convertir la consulta a un vector TF-IDF
    query_vector = vectorizer.transform([query_processed])

    # Calcular la similitud del coseno
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Obtener los índices de los 'top_k' documentos más similares
    top_k_indices = cosine_similarities.argsort()[::-1][:top_k]

    # Mostrar los resultados
    print(f"\n--- Resultados para la consulta: '{query}' ---")
    for i, idx in enumerate(top_k_indices):
        # Recuperar la información del DataFrame original
        doc_id = df.loc[idx, 'id']
        title = df.loc[idx, 'title']
        abstract_snippet = df.loc[idx, 'abstract'][:200] + "..." # Fragmento del resumen
        score = cosine_similarities[idx]

        print(f"\n{i+1}. ID: {doc_id} (Puntuación: {score:.4f})")
        print(f"   Título: {title}")
        print(f"   Resumen: {abstract_snippet}")

## Indexación BM25

In [ ]:
!pip install rank_bm25

In [ ]:
# Cargar los archivos necesarios
print("Cargando archivos...")
with open('bm25_model.pkl', 'rb') as f:
    bm25 = pickle.load(f)
df = pd.read_pickle('preprocessed_dataframe.pkl')
print("Archivos cargados.")

Cargando archivos...
Archivos cargados.


In [ ]:
# Preparar el corpus para BM25
# BM25 necesita una lista de listas de palabras.
tokenized_corpus = [doc.split(" ") for doc in df['texto_preprocesado']]

In [ ]:
from rank_bm25 import BM25Okapi

# Inicializar el modelo BM25
bm25 = BM25Okapi(tokenized_corpus)

# Guardar el modelo BM25
import pickle
with open('bm25_model.pkl', 'wb') as f:
    pickle.dump(bm25, f)

In [ ]:
# Función de Búsqueda BM25
def search_bm25(query: str, top_k: int = 10):
    """
    Busca con BM25 y muestra las puntuaciones normalizadas entre 0 y 1.
    """
    # Preprocesar y tokenizar la consulta
    query_processed = preprocesar_texto(query)
    tokenized_query = query_processed.split(" ")

    # Obtener las puntuaciones de BM25 para la consulta
    doc_scores = bm25.get_scores(tokenized_query)

    # Normalización Min-Max de las puntuaciones
    min_score = np.min(doc_scores)
    max_score = np.max(doc_scores)

    # Evitar división por cero
    if max_score - min_score > 0:
        normalized_scores = (doc_scores - min_score) / (max_score - min_score)
    else:
        normalized_scores = np.zeros_like(doc_scores)

    # Obtener los índices de los 'top_k' documentos más similares
    top_k_indices = np.argsort(normalized_scores)[::-1][:top_k]

    # Mostrar los resultados
    print(f"\n--- Resultados para la consulta: '{query}' ---")
    for i, idx in enumerate(top_k_indices):
        doc_id = df.loc[idx, 'id']
        title = df.loc[idx, 'title']
        abstract_snippet = df.loc[idx, 'abstract'][:200] + "..."
        score = normalized_scores[idx]

        print(f"\n{i+1}. ID: {doc_id} (Puntuación Normalizada: {score:.4f})")
        print(f"   Título: {title}")
        print(f"   Resumen: {abstract_snippet}")

## Generación de embeddings y vectorDB en Pinecone

In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar el DataFrame y el modelo de embeddings
df = pd.read_pickle('preprocessed_dataframe.pkl')
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar Embeddings
textos_a_incrustar = df['texto_preprocesado'].tolist()
embeddings = model.encode(textos_a_incrustar, show_progress_bar=True, convert_to_numpy=True)

# Añadir los embeddings al DataFrame
df['embedding'] = embeddings.tolist()
print(" Embeddings generados.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

 Embeddings generados.


In [ ]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Inicializa el cliente con API Key
pc = Pinecone(api_key="pinecone_api_key")

In [ ]:
index_name = "arxiv-search"
# --- Crear el índice en Pinecone
if index_name in pc.list_indexes().names():
    print(f"Eliminando índice existente: {index_name}...")
    pc.delete_index(index_name)

# Crear un nuevo índice
print(f"Creando nuevo índice: {index_name}...")
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

# Conectar al índice
index = pc.Index(index_name)
print("Conexión con el índice de Pinecone establecida.")

Creando nuevo índice: arxiv-search...
Conexión con el índice de Pinecone establecida.


In [ ]:
# Preparar y Cargar Datos a Pinecone
print("\nPreparando y cargando datos a Pinecone...")
batch_size = 128
for i in range(0, len(df), batch_size):

    batch = df.iloc[i:i+batch_size]

    vectors_to_upsert = []
    for _, row in batch.iterrows():
        vector_id = row['id']
        embedding = row['embedding']
        metadata = {
            "title": row['title'],
            "abstract": row['abstract']
        }
        vectors_to_upsert.append((vector_id, embedding, metadata))

    # Subir el lote al índice
    index.upsert(vectors=vectors_to_upsert)

print(f" ¡Carga completada! {index.describe_index_stats()['total_vector_count']} vectores subidos a Pinecone.")


Preparando y cargando datos a Pinecone...
 ¡Carga completada! 19968 vectores subidos a Pinecone.


In [ ]:
def search_pinecone(query: str, top_k: int = 10):
    """
    Busca en Pinecone y muestra los resultados.
    """
    # Generar el embedding para la consulta
    query_embedding = model.encode(query).tolist()

    # Realizar la búsqueda en Pinecone
    response = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    # Mostrar los resultados
    print(f"\n--- Resultados para la consulta: '{query}' ---")
    if not response['matches']:
        print("No se encontraron resultados.")
        return

    for i, match in enumerate(response['matches']):
        doc_id = match['id']
        score = match['score']
        title = match['metadata']['title']
        abstract_snippet = match['metadata']['abstract'][:200] + "..."

        print(f"\n{i+1}. ID: {doc_id} (Puntuación: {score:.4f})")
        print(f"   Título: {title}")
        print(f"   Resumen: {abstract_snippet}")

## RAG

In [ ]:
def retrieve_for_rag(query: str, top_k: int = 3) -> list:
    """
    Busca en Pinecone y RETORNA los metadatos de los top_k resultados.
    """
    query_embedding = model.encode(query).tolist()

    # Realizar la búsqueda en Pinecone
    response = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    # Extraer y devolver los metadatos de los resultados
    contexts = []
    if response['matches']:
        for match in response['matches']:
            contexts.append({
                "id": match['id'],
                "title": match['metadata']['title'],
                "abstract": match['metadata']['abstract']
            })
    return contexts

In [ ]:
from openai import OpenAI

In [ ]:
client_deepseek = OpenAI(api_key="api_key_deepseek", base_url="url")

In [ ]:
from IPython.display import display, Markdown

def generate_rag_response_deepseek(query: str):
    print(f"Buscando documentos relevantes para: '{query}'...")
    retrieved_docs = retrieve_for_rag(query, top_k=3)

    if not retrieved_docs:
        return "No pude encontrar documentos relevantes en la base de datos."

    context_str = ""
    for i, doc in enumerate(retrieved_docs):
        context_str += f"Documento {i+1} (ID: {doc['id']}):\n"
        context_str += f"Título: {doc['title']}\n"
        context_str += f"Resumen: {doc['abstract']}\n\n-----\n\n"

    prompt = f"""
    Use the following context to answer the question.
    If the answer is not contained in the context, respond with 'La información solicitada no se encuentra en los documentos proporcionados.'
    If you include mathematical formulas or equations, write them in LaTeX notation wrapped between double dollar signs ($$).
    First, provide a comprehensive answer to the question. Then, justify why each of the retrieved documents was relevant to the user's question.

    Context:
    {context_str}

    Question: {query}
    """

    print("Generando respuesta con DeepSeek...")
    try:
        response = client_deepseek.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[
                {"role": "system", "content": "You are an expert assistant in scientific papers. You answer questions based only on the provided context."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ocurrió un error al generar la respuesta con DeepSeek: {e}"

## Evaluación

In [ ]:
# Función evaluacion de búsqueda TF-IDF
def search_tfidf_eval(query: str, top_k: int = 10) -> list[str]:
    query_processed = preprocesar_texto(query)
    query_vector = vectorizer.transform([query_processed])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_k_indices = cosine_similarities.argsort()[::-1][:top_k]
    # Retorna la lista de IDs del DataFrame
    return df.loc[top_k_indices, 'id'].tolist()

# Función evaluacion de búsqueda BM25
def search_bm25_eval(query: str, top_k: int = 10) -> list[str]:
    query_processed = preprocesar_texto(query)
    tokenized_query = query_processed.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    top_k_indices = np.argsort(doc_scores)[::-1][:top_k]
    return df.loc[top_k_indices, 'id'].tolist()

# Función evaluacion de búsqueda Pinecone
def search_pinecone_eval(query: str, top_k: int = 10) -> list[str]:
    query_embedding = model.encode(query).tolist()
    response = index.query(vector=query_embedding, top_k=top_k, include_metadata=False)
    # Retorna la lista de IDs de la respuesta de Pinecone
    return [match['id'] for match in response['matches']]

In [ ]:
# Selecciona una consulta del archivo original
query_to_compare = "machine learning for particle physics"

# Obtener los top-10 IDs de cada modelo
tfidf_results = search_tfidf_eval(query_to_compare)
bm25_results = search_bm25_eval(query_to_compare)
pinecone_results = search_pinecone_eval(query_to_compare)

# tabla comparativa
comparison_df = pd.DataFrame({
    'Rank': range(1, 11),
    'TF-IDF': tfidf_results,
    'BM25': bm25_results,
    'Vector Search': pinecone_results
})

print("--- Tabla Comparativa de Resultados ---")
display(comparison_df)

# Calcular cuántos documentos aparecen en común
common_tfidf_bm25 = len(set(tfidf_results) & set(bm25_results))
common_tfidf_pinecone = len(set(tfidf_results) & set(pinecone_results))
common_bm25_pinecone = len(set(bm25_results) & set(pinecone_results))
common_all = len(set(tfidf_results) & set(bm25_results) & set(pinecone_results))

print(f"\n--- Coincidencias en el Top-10 ---")
print(f"Coincidencias entre TF-IDF y BM25: {common_tfidf_bm25}/10")
print(f"Coincidencias entre TF-IDF y Vector Search: {common_tfidf_pinecone}/10")
print(f"Coincidencias entre BM25 y Vector Search: {common_bm25_pinecone}/10")
print(f"Coincidencias entre los tres modelos: {common_all}/10")

--- Tabla Comparativa de Resultados ---


,Rank,TF-IDF,BM25,Vector Search
0,1,0704.3453,0704.3453,0707.2539
1,2,0705.0342,0707.0930,0707.4658
2,3,0708.1564,0704.0858,0706.2999
3,4,0704.3905,0708.1564,0708.2072
4,5,0705.2318,0704.3905,0706.0039
5,6,0707.3665,0707.0303,0707.2524
6,7,0705.0982,0708.1838,0705.1116
7,8,0707.4489,0705.3251,0706.3579
8,9,0707.0303,0704.1028,0705.0655
9,10,0707.3666,0707.1706,0706.2101



--- Coincidencias en el Top-10 ---
Coincidencias entre TF-IDF y BM25: 4/10
Coincidencias entre TF-IDF y Vector Search: 0/10
Coincidencias entre BM25 y Vector Search: 0/10
Coincidencias entre los tres modelos: 0/10


In [ ]:
# --- Resultados de TF-IDF
test_query = "machine learning for particle physics"
search_tfidf(test_query)


--- Resultados para la consulta: 'machine learning for particle physics' ---

1. ID: 0704.3453 (Puntuación: 0.5057)
   Título: An Adaptive Strategy for the Classification of G-Protein Coupled
  Receptors
   Resumen:   One of the major problems in computational biology is the inability of
existing classification models to incorporate expanding and new domain
knowledge. This problem of static classification models ...

2. ID: 0705.0342 (Puntuación: 0.4345)
   Título: Y(5S): What has been learned and what can be learned
   Resumen:   We present recent measurements of B and B^0_s production using data collected
on the Y(5S) resonance at CLEO and Belle. We also briefly discuss what can be
learned using sufficiently larger data sam...

3. ID: 0708.1564 (Puntuación: 0.3744)
   Título: Learning Phonotactics Using ILP
   Resumen:   This paper describes experiments on learning Dutch phonotactic rules using
Inductive Logic Programming, a machine learning discipline based on inductive
logical ope

In [ ]:
# --- Resultados BM25
test_query = "machine learning for particle physics"
search_bm25(test_query)


--- Resultados para la consulta: 'machine learning for particle physics' ---

1. ID: 0704.3453 (Puntuación Normalizada: 1.0000)
   Título: An Adaptive Strategy for the Classification of G-Protein Coupled
  Receptors
   Resumen:   One of the major problems in computational biology is the inability of
existing classification models to incorporate expanding and new domain
knowledge. This problem of static classification models ...

2. ID: 0707.0930 (Puntuación Normalizada: 0.8621)
   Título: Bayesian Learning of Neural Networks for Signal/Background
  Discrimination in Particle Physics
   Resumen:   Neural networks are used extensively in classification problems in particle
physics research. Since the training of neural networks can be viewed as a
problem of inference, Bayesian learning of neur...

3. ID: 0704.0858 (Puntuación Normalizada: 0.8225)
   Título: Lessons Learned from the deployment of a high-interaction honeypot
   Resumen:   This paper presents an experimental study and the 

In [ ]:
# --- Resultados Pinecone
test_query = "machine learning for particle physics"
search_pinecone(test_query)


--- Resultados para la consulta: 'machine learning for particle physics' ---

1. ID: 0707.2539 (Puntuación: 0.4227)
   Título: Three-dimensional Monte Carlo simulations of the quantum linear
  Boltzmann equation
   Resumen:   Recently the general form of a translation-covariant quantum Boltzmann
equation has been derived which describes the dynamics of a tracer particle in
a quantum gas. We develop a stochastic wave func...

2. ID: 0707.4658 (Puntuación: 0.4184)
   Título: Higgs Hadroproduction at Large Feynman x
   Resumen:   We propose a novel mechanism for the production of the Higgs boson in
inclusive hadronic collisions, which utilizes the presence of heavy quarks in
the proton wave function. In these inclusive react...

3. ID: 0706.2999 (Puntuación: 0.4171)
   Título: Searches in Photon and Jet States
   Resumen:   We present recent results from the Collider Detector at Fermilab (CDF) and D0
experiments using data from proton-antiproton collisions with sqrt(s) = 1.96
TeV at Run 

## Respuesta a las preguntas de evaluación e informe final

- ¿Cuáles documentos aparecen en común?
Los únicos modelos que comparten documentos en común son TF-IDF y BM25, comparten 4 documentos.
- ¿Qué diferencias hay en el ordenamiento?
Los documentos con id 0704.3453 aparecen primero tanto en TF-IDF, pero son los únicos que comparten su orden en el ranking, ya que los demás documentos que comparten, difieren en el orden del ranking entregado por TF-IDF y BM25

In [ ]:
query_rag_example = "What is the role of the Higgs boson decay in modern physics?"
rag_answer = generate_rag_response_deepseek(query_rag_example)

print("\n--- EJEMPLO DE RESPUESTA RAG ---")
display(Markdown(rag_answer))

Buscando documentos relevantes para: 'What is the role of the Higgs boson decay in modern physics?'...
Generando respuesta con DeepSeek...

--- EJEMPLO DE RESPUESTA RAG ---


**Answer:**  
The role of Higgs boson decay in modern physics is multifaceted, encompassing the validation of the Standard Model (SM), the exploration of beyond-the-Standard-Model (BSM) theories, and the determination of the Higgs boson's properties through precision measurements. Specific decay channels, such as invisible decays or photonic decays, provide critical insights into the Higgs's interactions with other particles and potential new physics. For example:  
1. **Testing BSM Scenarios**: Invisible decays (e.g., \( H \to A_H A_H \)) in the Littlest Higgs model with T-parity (Document 1) probe extensions of the SM, where deviations in branching ratios or decay widths signal new particles or symmetries.  
2. **Revealing Exotic Higgs Properties**: Fermiophobic Higgs bosons in Two Higgs Doublet Models (Document 2) exhibit enhanced \( H \to \gamma\gamma \) decays due to charged scalar loops, highlighting how decay patterns can expose non-SM Higgs sectors.  
3. **Precision Measurements**: The two-photon decay width (\( \Gamma_{\gamma\gamma} \)) of the Higgs (Document 3) is a key observable for determining its couplings, with experimental efforts aiming for sub-3% statistical errors to constrain SM predictions.  

---

**Relevance of Retrieved Documents:**  
- **Document 1 (ID: 0707.1591)**: Demonstrates how Higgs decays into heavy photons (\( A_H \)) in a BSM framework (Littlest Higgs model) can dominate for specific parameter ranges, illustrating the role of decays in testing theoretical extensions.  
- **Document 2 (ID: 0708.1939)**: Explores how charged scalar loops modify photonic decays (\( H \to \gamma\gamma \)), emphasizing the interplay between Higgs decay channels and new physics effects like additional Higgs doublets.  
- **Document 3 (ID: 0705.1259)**: Focuses on experimental strategies to measure the Higgs's two-photon decay width with high precision, underlining the importance of decay processes in empirical Higgs characterization.  

Each document highlights distinct aspects of Higgs decay physics, from theoretical BSM implications to experimental precision, collectively showcasing their centrality in modern particle physics research.


## **1. Análisis Comparativo de Modelos de Recuperación**

Se realizó una comparación entre los tres modelos de recuperación de información implementados (TF-IDF, BM25 y Búsqueda Vectorial) para una consulta de prueba. La siguiente tabla muestra los 10 documentos mejor clasificados por cada modelo.

| Rank | TF-IDF | BM25 | Vector Search |
| :--- | :--- | :--- | :--- |
| 1 | 0704.3453 | 0704.3453 | 0707.2539 |
| 2 | 0705.0342 | 0707.0930 | 0707.4658 |
| 3 | 0708.1564 | 0704.0858 | 0706.2999 |
| 4 | 0704.3905 | 0708.1564 | 0708.2072 |
| 5 | 0705.2318 | 0704.3905 | 0706.0039 |
| 6 | 0707.3665 | 0707.0303 | 0707.2524 |
| 7 | 0705.0982 | 0708.1838 | 0705.1116 |
| 8 | 0707.4489 | 0705.3251 | 0706.3579 |
| 9 | 0707.0303 | 0704.1028 | 0705.0655 |
| 10 | 0707.3666 | 0707.1706 | 0706.2101 |

### **Comparativa de modelos:**

  * **Similitud entre TF-IDF y BM25:** Ambos modelos, al estar basados en frecuencia de palabras clave, comparten cuatro documentos recuperados de su búsqueda. Documentos como `0704.3453`, `0708.1564` y `0704.3905` aparecen en ambos rankings, aunque en posiciones diferentes.

  * **Búsqueda Vectorial en Pinecone:** Este modelo difiere totalmente con los resultados obtenidos por los modelos TF-IDF y BM25. Esto se debe a la diferencia entre la búsqueda léxica y la búsqueda semántica. El modelo vectorial recupera documentos que están relacionados temáticamente con la consulta, incluso si no utilizan las mismas palabras clave, lo que le permite descubrir información relevante que los otros modelos pasarían por alto.

-----

## **2. Retrieval-Augmented Generation**

Se ejecutó el sistema RAG con la query "What is the role of the Higgs boson decay in modern physics?". El sistema recuperó tres documentos relevantes y generó la respuesta:

**"The role of Higgs boson decay in modern physics is multifaceted, encompassing the validation of the Standard Model (SM), the exploration of beyond-the-Standard-Model (BSM) theories, and the determination of the Higgs boson's properties through precision measurements. Specific decay channels, such as invisible decays or photonic decays, provide critical insights into the Higgs's interactions with other particles and potential new physics. For example:
Testing BSM Scenarios: Invisible decays (e.g., ( H \to A_H A_H )) in the Littlest Higgs model with T-parity (Document 1) probe extensions of the SM, where deviations in branching ratios or decay widths signal new particles or symmetries.
Revealing Exotic Higgs Properties: Fermiophobic Higgs bosons in Two Higgs Doublet Models (Document 2) exhibit enhanced ( H \to \gamma\gamma ) decays due to charged scalar loops, highlighting how decay patterns can expose non-SM Higgs sectors.
Precision Measurements: The two-photon decay width (( \Gamma_{\gamma\gamma} )) of the Higgs (Document 3) is a key observable for determining its couplings, with experimental efforts aiming for sub-3% statistical errors to constrain SM predictions."**

### **Evaluación de la Respuesta:**

1.  **Calidad y Coherencia:** La respuesta generada es de alta calidad, está bien estructurada y aborda la pregunta de manera integral.

2.  **Fidelidad al Contexto:** La respuesta hace referencia explícita a "Documento 1", "Documento 2" y "Documento 3" y utiliza la información contenida en ellos para construir su argumentación. La sección "Relevance of Retrieved Documents" confirma que el modelo ha entendido y utilizado correctamente el contexto.

3.  **Utilidad del Sistema RAG:** Este ejemplo demuestra el valor inmenso de la arquitectura RAG. En lugar de devolver una simple lista de documentos que el usuario tendría que leer y sintetizar por su cuenta, el sistema realiza este trabajo, entregando una respuesta directa, concisa y basada en evidencia.

-----

## **3. Conclusiones Generales**

La implementación y evaluación del sistema de recuperación de información demuestran que no existe un único modelo definitivo que se deba aplicar, ya que cada uno aborda distintos enfoques y se pueden aplicar en distintas áreas con requerimientos específicos.

La arquitectura RAG es una solución más completa, ya que combina la precisión de la recuperación de información con el poder de síntesis de los modelos de lenguaje. Este enfoque híbrido es capaz de proporcionar respuestas contextualizadas y de gran utilidad.